In [13]:
import sys; sys.path.append("../src")
from direct import Direct
from optimisation import conds
import numpy as np, matplotlib.pyplot as plt

In [14]:
# initial and final states
x0 = [0,0,0,0]
xf = [1,0,0,0]

In [15]:
# instantiate direct segment
seg = Direct(x0, xf)

In [21]:
for cond in conds:
    try:
        a = np.load("../data/min_times/" + str(cond) + ".npy")
    except:
        continue
    print(*cond, a)

[-1, 0, 0, 0] [0, 0, 0, 0] 4.555161842684519
[-1, 0, 0, 0] [1, 0, 0, 0] 5.528949956195733
[-1, 0, 0, 0] [-1, 0, 3.141592653589793, 0] 6.156613044613905
[-1, 0, 0, 0] [0, 0, 3.141592653589793, 0] 4.810676389792809
[-1, 0, 0, 0] [1, 0, 3.141592653589793, 0] 5.34646217494403
[0, 0, 0, 0] [-1, 0, 0, 0] 4.555161842684519
[0, 0, 0, 0] [1, 0, 0, 0] 4.555161842684517
[0, 0, 0, 0] [-1, 0, 3.141592653589793, 0] 6.729461011662347
[0, 0, 0, 0] [0, 0, 3.141592653589793, 0] 6.656613044613476
[0, 0, 0, 0] [1, 0, 3.141592653589793, 0] 4.310676389792809
[1, 0, 0, 0] [-1, 0, 0, 0] 5.528949956195732
[1, 0, 0, 0] [0, 0, 0, 0] 0.5000000000022201
[1, 0, 0, 0] [-1, 0, 3.141592653589793, 0] 6.826386803881379
[1, 0, 0, 0] [0, 0, 3.141592653589793, 0] 6.229461034139615
[1, 0, 0, 0] [1, 0, 3.141592653589793, 0] 6.156613044613904
[-1, 0, 3.141592653589793, 0] [-1, 0, 0, 0] 6.656613044613108
[-1, 0, 3.141592653589793, 0] [0, 0, 0, 0] 6.729461011662347
[-1, 0, 3.141592653589793, 0] [1, 0, 0, 0] 6.3263868038814115
[

In [22]:
np.array([[0,1,2,3],[1,2,3,4,5]])

array([list([0, 1, 2, 3]), list([1, 2, 3, 4, 5])], dtype=object)